In [1]:
"""

LINKS:

- https://github.com/sagemath/sage/issues/38456
- https://mathoverflow.net/questions/271932/formula-for-the-frobenius-schur-indicator-of-a-finite-group
- https://math.stackexchange.com/questions/832173/structure-of-g-invariant-bilinear-forms-over-finite-fields

""";

In [2]:
"""
NOTES:

For a unitary change-of-basis for a rep'n \rho of S_n over finite fields, we need:

1) find a S_n-invariant symmetric bilinear form w/ associated matrix U
2) factor U = A^*A, where * denotes conjugate-transpose w.r.t. x |--> x^q conjugation

1 sol'n) let U be unknown in variables over a polynomial ring over GF(q^2). use linear algebra to solve \rho(g)^T U \overline{\rho(g)} == U
2 sol'n) find the upper triangular part Up of the LU decomposition of the matrix, multiply Up transpose by the square root of Up diagonal to get A. then A^*UA = D, diagonal. factor D = RR^*, and RA^{-1} is the change-of-b

Then \tilde{\rho}(g) = A\rho(g)A.inverse() is unitary for all g \in G.

""";

In [1]:
def invariant_symmetric_bilinear_matrix(partition):
    """
    Computes the matrix of a S_n-invariant symmetric bilinear form.
    Sets up and solves system of linear equations based on writing U as an unknown in polynomial ring generators. 
    The equations are \rho(g)^T*U*\overline{\rho(g)} = \lambda_g*U where \lambda_g = \det(\rho(g))\overline{\det(\rho(g))}.
    The variables for U can be extracted to yield a matrix over GF(q^2) for each g.
    These are stacked to get the overall system, and we find the one dim'l null space to get a solution vector, and format as a matrix.
    NOTE: one could also form the Kroenecker products \rho(g) \otimes \rho(g)^{-1 T} to explicitly obtain the system.
    NOTE: the solution space is typically 1 dim'l, except in "modular" cases where p divides |G|, and there is multiplicity in the decomposition factors.
    """
    # Define the group G and its rep'n as a Specht module, dimension
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    d_rho = specht_module.dimension()
    # Initialize U as a matrix of variables over GF(q^2)
    R = PolynomialRing(F, 'u', d_rho**2)
    U_vars = R.gens()  # List of variable generators for U
    U = matrix(R, d_rho, d_rho, U_vars)  # U is a d_rho x d_rho matrix of variables
    # for each generator of G, form the augmented system 
    def augmented_matrix(g):
        #form the matrix equation \rho(g)^T*U*\overline{\rho(g)} = \lambda_g * U
        #lambda_g = rho_g.det()*rho_g.det()**q
        #note: \lambda_g isn't necessary. probably only relates to sesquilinear forms
        rho_g = rho(g)
        equation_matrix = rho_g.transpose()*U*rho_g.conjugate() - U
        # Initialize a list to hold rows of the augmented system
        augmented_system = []
        # Extract coefficients for each linear equation in the matrix
        for i in range(d_rho):
            for j in range(d_rho):
                # Get the (i, j) entry of the equation matrix, which is a linear combination of the u variables
                linear_expression = equation_matrix[i, j]
                # Extract the coefficients of each u_k in the linear expression
                row = [linear_expression.coefficient(u) for u in U_vars]
                # Append the row to the augmented system
                augmented_system.append(row)
        # Convert the augmented system to a matrix
        return matrix(F, augmented_system)
    #stack linear systems for each g in G
    total_system = matrix(F,0,d_rho**2)
    for g in G:
        total_system = total_system.stack(augmented_matrix(g))
    #compute the null space of the overall matrix
    null_space = total_system.right_kernel()
    #return a d_rho x d_rho matrix over GF(q^2) from the 1 dim'l null space given as vector
    U_mats = [matrix(F,d_rho,d_rho,b) for b in null_space.basis()]
    return U_mats

In [6]:
#define a general bi/bilinear form with the matrix U
def bilinear_form(x,y,U):
    return x.transpose()*U*y

In [13]:
#ensure the resulting form is G-invariant, symmetric, bilinear by symbolic verification
def check_form_properties(q,partition):
    #define the representation matrix corresponding to q, partition
    SGA = SymmetricGroupAlgebra(GF(q^2),sum(partition))
    SM = SGA.specht_module(partition)
    rho = SM.representation_matrix
    d_rho = SM.dimension()
    G = SGA.group()

    #define variables as polynomial generators
    R_xy = PolynomialRing(GF(q^2), d_rho, var_array='x,y')
    x = matrix([R_xy.gens()[2*i] for i in range(d_rho)]).transpose()
    y = matrix([R_xy.gens()[2*i+1] for i in range(d_rho)]).transpose()
    R_xy_lambda = PolynomialRing(R_xy,'lambda')
    lambda_ = R_xy_lambda.gens()[0]

    #compute the bilinear form matrix. coerce over polynomial ring
    U_mats = invariant_symmetric_bilinear_matrix(partition)
    if len(U_mats) > 1:
        print("Space of G-invariant symmetric bilinear forms has dimension > 1 for la=",partition)
        print("Dimension of space=",len(U_mats))
    U_mat = U_mats[0]
    U_form = matrix(R_xy_lambda,U_mat)
    
    #check symmetric property
    symmetric = bilinear_form(x,y,U_form) == bilinear_form(y,x,U_form)
    
    #check G-invariance property
    G_invariant = all(bilinear_form(rho(g)*x,rho(g)*y,U_form) == bilinear_form(x,y,U_form) for g in G)
    
    #check bilinear property. ISSUE: lambda_^q is a power of the ring generator, i.e. doesn't simplify.
    first_arg = bilinear_form(lambda_*x,y,U_form) == lambda_*bilinear_form(x,y,U_form)
    second_arg = bilinear_form(x,lambda_*y,U_form) == lambda_*bilinear_form(x,y,U_form) #need to amend for conjugation
    bilinear = first_arg and second_arg

    return symmetric and G_invariant and bilinear

In [8]:
#print the checks for pairs (p,la) where p is a prime and la is a partition of n
for p in Primes()[:3]:
    print(f"p={p}")
    for n in range(3,6):
        print([(la,check_form_properties(p,la)) for la in Partitions(n)])
    print("---------")

p=2
[([3], True), ([2, 1], True), ([1, 1, 1], True)]
[([4], True), ([3, 1], True), ([2, 2], True), ([2, 1, 1], True), ([1, 1, 1, 1], True)]
Space of G-invariant symmetric bilinear forms has dimension > 1 for la= [3, 1, 1]
Dimension of space= 2
[([5], True), ([4, 1], True), ([3, 2], True), ([3, 1, 1], True), ([2, 2, 1], True), ([2, 1, 1, 1], True), ([1, 1, 1, 1, 1], True)]
---------
p=3
[([3], True), ([2, 1], True), ([1, 1, 1], True)]
[([4], True), ([3, 1], True), ([2, 2], True), ([2, 1, 1], True), ([1, 1, 1, 1], True)]
[([5], True), ([4, 1], True), ([3, 2], True), ([3, 1, 1], True), ([2, 2, 1], True), ([2, 1, 1, 1], True), ([1, 1, 1, 1, 1], True)]
---------
p=5
[([3], True), ([2, 1], True), ([1, 1, 1], True)]
[([4], True), ([3, 1], True), ([2, 2], True), ([2, 1, 1], True), ([1, 1, 1, 1], True)]
[([5], True), ([4, 1], True), ([3, 2], True), ([3, 1, 1], True), ([2, 2, 1], True), ([2, 1, 1, 1], True), ([1, 1, 1, 1, 1], True)]
---------


In [9]:
#NOTE: for q=2, la=[3,1,1] we get a two dim'l space
invariant_symmetric_bilinear_matrix([3,1,1])

[
[1 1 1 1 1 0]  [0 0 0 0 0 1]
[1 1 1 1 0 1]  [0 0 0 0 1 0]
[1 1 1 0 1 1]  [0 0 0 1 0 0]
[1 1 0 1 1 1]  [0 0 1 0 0 0]
[1 0 1 1 1 1]  [0 1 0 0 0 0]
[0 1 1 1 1 1], [1 0 0 0 0 0]
]

In [10]:
#to explain q=2, la=[3,1,1] note that the one of the composition factors (the irreducible quotients from any composition series)
#has multiplicity 2. we see this by looking at the Brauer character of the composition factor, and see (1,1,1) appears twice.
S5 = SymmetricGroup(5)
SGA = S5.algebra(GF(2))
for la in Partitions(5):
    M = SGA.specht_module(la)
    print(la, [V.brauer_character() for V in M.composition_factors()])

[5] [(1, 1, 1)]
[4, 1] [(4, 1, -1)]
[3, 2] [(4, -2, -1), (1, 1, 1)]
[3, 1, 1] [(1, 1, 1), (4, -2, -1), (1, 1, 1)]
[2, 2, 1] [(1, 1, 1), (4, -2, -1)]
[2, 1, 1, 1] [(4, 1, -1)]
[1, 1, 1, 1, 1] [(1, 1, 1)]


In [20]:
#print the matrix associated to the bilinear form for each partition
#note q needs to be large enough. p \nmid n! seems sufficient
for la in Partitions(5):
    U_mats = invariant_symmetric_bilinear_matrix(la)
    if len(U_mats) > 1:
        print("space of G-invariant symmetric bilinear forms has dimension greater than 1")
    print("la =",la)
    print("q =",q)
    print(U_mats[0])
    print("--------")

la = [5]
q = 7
[1]
--------
la = [4, 1]
q = 7
[1 4 4 4]
[4 1 4 4]
[4 4 1 4]
[4 4 4 1]
--------
la = [3, 2]
q = 7
[1 4 4 2 5]
[4 1 2 4 2]
[4 2 1 4 2]
[2 4 4 1 4]
[5 2 2 4 1]
--------
la = [3, 1, 1]
q = 7
[1 5 2 5 2 0]
[5 1 5 5 0 2]
[2 5 1 0 5 2]
[5 5 0 1 5 5]
[2 0 5 5 1 5]
[0 2 2 5 5 1]
--------
la = [2, 2, 1]
q = 7
[1 5 2 2 5]
[5 1 5 5 5]
[2 5 1 5 5]
[2 5 5 1 5]
[5 5 5 5 1]
--------
la = [2, 1, 1, 1]
q = 7
[1 2 5 2]
[2 1 2 5]
[5 2 1 2]
[2 5 2 1]
--------
la = [1, 1, 1, 1, 1]
q = 7
[1]
--------


In [14]:
#for u in GF(q), we can factor as u=aa^* using gen. z and modular arithmetic
def conj_square_root(u):
    if u == 0:
        return 0  # Special case for 0
    z = F.multiplicative_generator()
    k = u.log(z)  # Compute discrete log of u to the base z
    if k % (q+1) != 0:
        raise ValueError("Unable to factor: u is not in base field GF(q)")
    return z ** ((k//(q+1))%(q-1))

In [39]:
#user the upper part of the LU decomposition, factor the diagonal
def base_change_hermitian(U):
    Up = U.LU()[2] #upper triangluar part of LU decompostion
    D = Up.diagonal() #diagonal of upper triangular part
    A = ~Up * matrix.diagonal([d.sqrt() for d in D]) #transpose of upper triangular part * square root of diagonal
    diag = (A.H * U * A).diagonal() #A_star*U*A == D, a diagonal
    factor_diag = diagonal_matrix([conj_square_root(d) for d in diag]) #factor diagonal with conjugate square root
    return factor_diag*A.inverse()

In [24]:
#define the representation matrix corresponding to q, partition
#define generators of multiplicative groups of GF(q) and GF(q^2), w and z respectively
q = 7; la = [2,1,1]
SGA = SymmetricGroupAlgebra(GF(q**2),sum(la))
SM = SGA.specht_module(la)
rho = SM.representation_matrix
d_rho = SM.dimension()
G = SGA.group()
F = GF(q**2)
z = F.gen()

In [25]:
#compute the S_n-invariant symmetric bilinear form associated to (q,la)
U = invariant_symmetric_bilinear_matrix(la)[0]; U

[1 5 2]
[5 1 5]
[2 5 1]

In [38]:
U.LU()

(
[1 0 0]  [1 0 0]  [1 5 2]
[0 1 0]  [5 1 0]  [0 4 2]
[0 0 1], [2 4 1], [0 0 3]
)

In [26]:
#compute the factorization U = AA^*
A = base_change_hermitian(U)
U == A.H*A

True

In [27]:
#construct the unitary representations as \tilde{\rho}(g) = A^*\rho(g)A^*.inverse()
all_unitary = True
for g in G:
    tilde_rho_g = A*rho(g)*A.inverse()
    unitary_check = tilde_rho_g.H*tilde_rho_g == identity_matrix(F,d_rho)
    all_unitary = all_unitary and unitary_check
print(all_unitary)

True


In [28]:
#define the Fourier coefficient at the rep'n specht_module corresponding to partition
def hat(g,partition,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        U = invariant_symmetric_bilinear_matrix(partition)[0]
        A = base_change_hermitian(U)
        sqrt_unitary_factor = sqrt(F(specht_module.dimension()/G.cardinality()))
        return sqrt_unitary_factor*A*rho(g)*A.inverse()
    else:
        return rho(g)

In [29]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
def dft(unitary=False):
    fourier_transform = [[x for partition in Partitions(G.degree()) for x in hat(g, partition, unitary).list()] for g in G]
    if unitary:
        dft_matrix = matrix(F,fourier_transform).transpose()
        sign_diag = (dft_matrix*dft_matrix.H).diagonal()
        factor_diag_inv = diagonal_matrix([~conj_square_root(d) for d in sign_diag])
        return factor_diag_inv*dft_matrix
    else:
        return matrix(fourier_transform).transpose()

In [33]:
#compute the unitary DFT
unitary_dft = dft(unitary=True); print(list(unitary_dft))

[(2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2, 2*z2), (1, 1, 4, 4, 4, 4, 6, 6, 3, 3, 3, 3, 3, 3, 4, 4, 0, 0, 3, 3, 4, 4, 0, 0), (0, 0, 3*z2, z2, 3*z2, z2, 0, 0, 4*z2, 6*z2, 4*z2, 6*z2, 3*z2, z2, 4*z2, 6*z2, 5*z2, 2*z2, 3*z2, z2, 4*z2, 6*z2, 5*z2, 2*z2), (0, 0, 0, z2, 0, z2, 0, 0, 0, 6*z2, 0, 6*z2, 0, z2, 0, 6*z2, z2, 6*z2, 0, z2, 0, 6*z2, z2, 6*z2), (0, 0, 4*z2 + 3, 4*z2 + 3, 6*z2 + 1, 6*z2 + 1, 0, 0, 4*z2 + 3, 4*z2 + 3, 6*z2 + 1, 6*z2 + 1, 3*z2 + 4, 3*z2 + 4, 3*z2 + 4, 3*z2 + 4, 2*z2 + 5, 2*z2 + 5, z2 + 6, z2 + 6, z2 + 6, z2 + 6, 5*z2 + 2, 5*z2 + 2), (1, 5, 3, 1, 1, 2, 1, 5, 3, 1, 1, 2, 3, 1, 3, 1, 4, 4, 1, 2, 1, 2, 4, 4), (0, 5, 0, 1, 5, 1, 0, 5, 0, 1, 5, 1, 0, 1, 0, 1, 1, 1, 5, 1, 5, 1, 1, 1), (0, 0, 0, 0, 6*z2 + 1, 6*z2 + 1, 0, 0, 0, 0, 6*z2 + 1, 6*z2 + 1, 0, 0, 0, 0, 6*z2 + 1, 6*z2 + 1, z2 + 6, z2 + 6, z2 + 6, z2 + 6, z2 + 6, z2 + 6), (0, 5, 0, 5, 1, 1, 0, 5, 0, 5, 1, 1, 0, 5, 0, 5, 1, 1, 

In [32]:
#verify the resulting DFT is unitary
unitary_dft*unitary_dft.H == identity_matrix(SGA.group().cardinality())

True